# Exploration of Merging Data Files

This notebook was built to explore how to best merge the Cisco Umbrella 1m Domains file and the Bambenek Consulting DGA High-Confidence Feed.

In [3]:
# Libraries
import zipfile
import numpy as np
import pandas as pd
import re
import os
import time
import sys
import lstm_binary
import imp

In [5]:
merged_csv = '/home/nscsekhar/team_cyber/data/local/staging/merged-2018-11-24.csv'
df = pd.read_csv(merged_csv)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df['domain']
Y = df['dga']

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=23)

In [10]:
imp.reload(lstm_binary)
train_model = lstm_binary.LSTMBinary()
train_model.train(X_train, Y_train)

Epoch 1/5
1490995/1490995 [==============================] - 173s 116us/step - loss: 0.1663
Epoch 2/5
1490995/1490995 [==============================] - 173s 116us/step - loss: 0.0885
Epoch 3/5
1490995/1490995 [==============================] - 173s 116us/step - loss: 0.0810
Epoch 4/5
1490995/1490995 [==============================] - 173s 116us/step - loss: 0.0623
Epoch 5/5
1490995/1490995 [==============================] - 173s 116us/step - loss: 0.0726


In [11]:
PROJECT_ROOT = "./"
BINARY_TOKENIZER_FILE = PROJECT_ROOT + "saved_models/binary_tokenizer.pkl"
BINARY_CATEGORIES_FILE = PROJECT_ROOT + "saved_models/binary_categories.pkl"
BINARY_MODEL_JSON = PROJECT_ROOT + "saved_models/binary_LSTM.json"
BINARY_MODEL_H5 = PROJECT_ROOT + "saved_models/binary_LSTM.h5"

train_model.save(BINARY_TOKENIZER_FILE, BINARY_MODEL_JSON, BINARY_MODEL_H5)

MODEL SAVED TO DISK!


In [12]:
testmodel = lstm_binary.LSTMBinary()
testmodel.load(BINARY_TOKENIZER_FILE, BINARY_MODEL_JSON, BINARY_MODEL_H5)

SAVED BINARY MODEL IS NOW LOADED!


In [13]:
urllist = ["www.google.com", "www.netflix.com", "plvklpgwivery.com"]
urltypes = testmodel.predict(urllist)
print("URL type:", urltypes)

['www.google.com', 'www.netflix.com', 'plvklpgwivery.com']
URL type: [[0]
 [0]
 [1]]


In [14]:
urltypes = testmodel.predict(X_test)

478510                                meteo-saintetienne.com
215684                                   jdloorwe4owunkp.com
1528784                          uqmy-media.s3.amazonaws.com
426127                                  axyljkhvvxpucg.co.uk
257901                                      pregqhhhstko.com
1568032                        updates2.safer-networking.org
1855549                                  519188313.localhost
1060218                                   down111.bizport.cn
1536272                                       weaksecond.net
448272                                   qnokajvf.emltrk.com
1242820            blogpfthumb-phinf.pstatic.net.edgekey.net
1300299                            bittorrent-2.hs.llnwd.net
1547445                                             fmca.com
865139                                       adaptedmind.com
835457                                        m38.ehmmuub.cc
1105728                                    scholar.google.ca
1257913                 

In [33]:
print((urltypes.reshape))

(372749, 1)


In [34]:
print(len(Y_test.values.shape))

1


In [26]:
print(urltypes[:10])
print(Y_test.values[:10])

[[0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
[0 1 0 1 1 0 0 0 0 0]


In [36]:
np.count_nonzero(urltypes.flatten() == Y_test.values)

365587